In [ ]:
import optuna
import pandas as pd
import numpy as np
import yaml
import pickle
import torch
import os

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import tensor, cat, save, load, optim, nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from src.models.resnet18_model import ResNet18

from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend

import src.study_handler as sh, get_shadow_signals
from src.utils import print_yaml

In [ ]:
# -------------#
#  Set device  #
# -------------#
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

In [ ]:
#----------------------#
#  Load Model Signals  #
#----------------------#
audit_signals_folder = "20251117_b2919d4746"
metadata, rescaled_target_logits, rescaled_shadow_model_logits, shadow_models_in_mask, target_in_mask, audit_data_indices = loadAudit(audit_signals_folder)

In [ ]:
#------------------------------#
#  Load Study & Update Config  #
#------------------------------#
config = None
with open("./study.yaml") as file:
    config = yaml.safe_load(file)
    
print_yaml(config)
    
data_cfg = config['data']

In [ ]:
#------------------------#
#  Select Shadow Models  #
#------------------------#
sm_amount = config['fbd_study']['sm_amount']
shadow_logits, shadow_inmask = get_shadow_signals(rescaled_shadow_model_logits, shadow_models_in_mask, sm_amount)


In [ ]:
#--------------------------------------------#
#  Calculate Vulnerability score using RMIA  #
#--------------------------------------------#
rmia_scores = None #rmia_vectorized(rescaled_shadow_model_logits, shadow_models_in_mask, rescaled_target_logits, ...)

In [ ]:
#----------------------------------#
#  Normalize Vulnerability Scores  #
#----------------------------------#
percentile = 2
norm_scores = percentile_score_normalization(rmia_scores, percentile)

In [ ]:
#-------------------#
#  Prepare dataset  #
#-------------------#
from src.dataset_handler import processDataset, loadDataset

trainset, testset = loadDataset(data_cfg)

# Will split the dataset to use the same in indices as the baseline target model
train_dataset, test_dataset, train_indices, test_indices = processDataset(data_cfg, trainset, testset, in_indices_mask=target_in_mask)

In [ ]:
# ------------------------#
#        Run study        #
# ------------------------#
import src.save_load as sl

def run_optimization(config):
    study_cfg = config['fbd_study']
    
    metadata = sl.buildStudyMetadata(study_cfg, config['data'])
    _, save_path = sl.saveStudy(metadata, savePath=study_cfg['root'])
    
    journal_path = os.path.join(save_path, "journal.log")
    storage = JournalStorage(JournalFileBackend(file_path=journal_path))
    
    study = optuna.create_study(
        study_name=study_cfg["study_name"],
        storage=storage,
        load_if_exists=True,
        directions=["maximize", "minimize"]
    )
    study.optimize(lambda trial: sh.fbd_objective(trial, norm_scores, trainset,
                                                testset, config, shadow_logits,
                                                shadow_inmask ), n_trials=study_cfg["trials"]
    )
    
    
    print(f"Study '{study_cfg['study_name']}' completed. Best value: {study.best_value}, params: {study.best_params}")
    
    df = study.trials_dataframe()
    df.to_csv(os.path.join(save_path, "results.csv"), index=False)
    print(f"📄 Results saved to {os.path.join(save_path, 'results.csv')}")

    return study

study = None
if config is not None:
    study = run_optimization(config)

In [ ]:
if study is not None:
    optuna.visualization.plot_pareto_front(study)
    optuna.visualization.plot_param_importances(study)